# This version

Trying to implement a version about CTC with 3 hidden layer and combine with RNN

***Summary***
1. Try Deep-LSTM by lasagne
- Try dropout layer
- [CTC Problem] prefix search decoding? -> blank probability threshold set at 99.99% at paper
- Try phoneme base instead of char base

***Todo***
1. ~~data normalize~~
2. CTC correct?
- Try phoneme but not character based
1. ~~Using oop module~~
- ~~Using CTC with log domain and recurrent relationship~~
- ~~Add hidden layer before RNN~~
- ~~Compile and update and learning~~
- ~~func check_label_error~~
- ~~data processing to char based~~
    - ~~preprocess of data~~
    - ~~remove_blank(predict)~~
    - ~~remap_back( predict )~~
    - ~~change to tri MFCC only~~
- ~~Try CTC with character based data (data preprocess) (our data)~~
- ~~Check CTC cost correctness~~
- ~~Check CTC with y need to unmap or not~~
- ~~Add Nestrove momentum~~
- ~~Add minibatch~~
- ~~Add bidirectional LSTM/RNN~~

***Further***
1. ~~Add Bidirectional LSTM/RNN~~
2. ~~Make mini batch~~
- Try phoneme based model

***Problem need to understand about RNN***
1. [Phd thesis Training RNN , Hessian free method](http://www.cs.utoronto.ca/~ilya/pubs/ilya_sutskever_phd_thesis.pdf)
- [Does Theano do automatic unfolding for BPTT?](http://stackoverflow.com/questions/24431621/does-theano-do-automatic-unfolding-for-bptt)
- [Github, General purpose Hessian-free optimization in Theano](https://github.com/boulanni/theano-hf)
- [Theano scan](http://deeplearning.net/software/theano/library/scan.html#theano.scan)
- []()

***Problem need to understand about CTC***
1. [CTC paper](ftp://ftp.idsia.ch/pub/juergen/icml2006.pdf)

***Important References***
1. [CTC cost use in our version](https://github.com/mohammadpz/CTC-Connectionist-Temporal-Classification/blob/5b4d4be19805d7795a4293b9c270ef7bf5fafbfc/ctc_cost.py)
    - [Discussion](https://github.com/mohammadpz/CTC-Connectionist-Temporal-Classification/issues/1)
- How to connect multiple recurrent layers,https://github.com/craffel/nntools/issues/9
- https://github.com/craffel/nntools/issues/11
- refactor recurrent, update examples, add tests, https://github.com/craffel/nntools/pull/27
- example of deep lstm and penn tree,https://github.com/skaae/nntools/blob/combine/examples/lstm.py

***Github***
1. [Hessian theano](https://github.com/boulanni/theano-hf/blob/master/hf_examples.py)
- [Vanilla RNN](https://github.com/mohammadpz/Recurrent-Neural-Networks)
- [blocks example, reverse words](https://github.com/mila-udem/blocks-examples/tree/master/reverse_words)
- [rnn-ctc, basics ctc](https://github.com/rakeshvar/rnn_ctc/blob/master/ctc.py)
- [Lasagne nntools](https://github.com/craffel/nntools/tree/recurrent)
- [Theanets, RNN tools](https://github.com/lmjohns3/theanets/blob/master/theanets/layers/recurrent.py)
- [LSTM benchmarks example](https://github.com/craffel/lstm_benchmarks/blob/master/lasagne/experiment.py)

In [1]:
import sys
sys.path.append("../../../../nntools")

In [2]:
import theano
import theano.tensor as T
import numpy as np
from theano_toolkit import utils as U
from theano_toolkit import updates
from theano.printing import Print
from theano_toolkit.parameters import Parameters

import lasagne
from lasagne.layers import *
# from lasagne.layers import RecurrentLayer, InputLayer, DenseLayer,\
#     NonlinearityLayer, ReshapeLayer, EmbeddingLayer
# from lasagne.layers.recurrent import *

from time import time

import ctc_cost_2

import cPickle
import sys
sys.setrecursionlimit(100000)


Using gpu device 0: GeForce GTX 980
/home/pika/nntools/lasagne/init.py:86: UserWarning: The uniform initializer no longer uses Glorot et al.'s approach to determine the bounds, but defaults to the range (-0.01, 0.01) instead. Please use the new GlorotUniform initializer to get the old behavior. GlorotUniform is now the default for all layers.
  warnings.warn("The uniform initializer no longer uses Glorot et al.'s "


# Data part

## read data

In [3]:
### DataProcessing 

'''
[ INPUT  ] : model { wav, NFCC, ... }
[ OUTPUT ] : (Training_data, Validation_data, Testing_Data)

- Training_data : x, y { .astype(np.float32)
- Validation_data : x, y { .astype(np.float32)
- Testing_Data : ( x, id )

-- x format : 
---- MFCC : [ 39-vector ] - 
---- FBank : [ 69-vector ]
---- quadmfcc : [ 39*(1+1+1) - vector ] !!! Wron---- FBank : [ 69-vector ]
---- quadmfcc : [ 39*(1+1+1) - vector ] !!! Wrong implementation
g implementation
---- hexamfcc : [ 39*(4+1+4) - vector ]
---- hexaFbank : [ 69*(4+1+4) - vector ]

-- y format :

---- Now we use 39-phonemes for all : [ 0 0 0 ... 1 .... 0 0 0 ]  as a number of 39
---- But answer we get from ./label is [ 48-vector]
'''


'''
input:
    MFCC: 39,2-1-2
output:
    state label: 1943
'''

'''
[ Input  ] :
    - mfcc (39*(2+1+2))

[ Output ] :
    - maxi (1943-D)
    
'''

from itertools import repeat
import numpy as np
import random
# from sklearn import preprocessing

# Parameter (note: switch /final/MLDS_Final_Data/ .. if u want
# MAPPING_FILE = '/home/pika/MLDS_2015/final/MLDS_HW1_RELEASE_v1/phones/48_39.map'
MFCC_TRAINING_ARK = '/home/pika/MLDS_2015/final/MLDS_Final_Data/mfcc/train_nor.ark'

# MAXI_TRAINING_LABEL = '/home/pika/MLDS_2015/final/MLDS_HW1_RELEASE_v1/state_label/train.lab'
# NORMAL_TRAINING_LABEL = '/home/pika/MLDS_2015/final/MLDS_HW1_RELEASE_v1/label/train.lab'

sentence_data = '/home/pika/MLDS_2015/final/MLDS_Final_Data/sentence/train_mod.set'

MFCC_TRAINING_SPEAKER_ARK = '/home/pika/MLDS_2015/final/MLDS_Final_Data/mfcc/train_nor_speaker.ark'
MFCC_VALID_SPEAKER_ARK = '/home/pika/MLDS_2015/final/MLDS_Final_Data/mfcc/valid_nor_speaker.ark'



# Dataset 
def DNNDataset( inputMode ) :
    trains = {}
    valids = {}
    test_data = {}
    x_train = []
    x_valid = []
    y_train = []
    y_valid = []
    total_result = []
    
    slide_ration = 0.1
    
    sentense = {}
    char_map = {}
    char_unmap = {}
    char_count = 0
    
    with open(sentence_data) as f:
        for lines in f:
            frames = lines.split(',',1)
            frames[1] = frames[1].lower()
            if len(frames[1])<2: # empty sentences
                prob = frames[0]
                print prob
                continue
            for cha in frames[1][:-2]: # skip "\n" ending punctuation
                if cha in ',.!?;:-"':
                    continue
                if cha not in char_map:
                    char_map[cha] = char_count
                    char_unmap[char_count] = cha
                    char_count += 1
                    
    char_map["-"] = char_count # for blank, different with space
    char_unmap[char_count] = "-"
#     print char_map
    
    with open(sentence_data) as f:
        for lines in f:
            frames = lines.split(',',1)
            frames[1] = frames[1].lower()
            if len(frames[1])<2: # empty sentences
                prob = frames[0]
                continue
            sentense[ frames[0] ] = []
            for cha in frames[1][:-2]:
                if cha in ',.!?;:"':
                    continue
                if cha in '-':
                    cha = " "
#                 sentense[ frames[0] ].append( char_map['-'] ) # append blank before each label
                sentense[ frames[0] ].append( char_map[cha] )
#             sentense[ frames[0] ].append( char_map['-'] ) # append blank in the end

    
    
    
    # X(INPUT) ---------------------------------------
    if inputMode == "mfcc":
        
        # speaker_window -(map)-> mfcc[39]
        monoTrains = {}
        with open(MFCC_TRAINING_ARK) as f:
            for lines in f:
                frames = lines.split(' ')
                frame2float = [ float(x) for x in frames[1:] ]
                monoTrains[frames[0]] = frame2float
#                 break
        # Initialize : L2 L1 S R1 R2
        zeroFrame = list(repeat(0, 39))
        prevframeL1 = [ float(x) for x in zeroFrame[:] ]
#         prevframeL2 = [ float(x) for x in zeroFrame[:] ]
        prevframeR1 = [ float(x) for x in zeroFrame[:] ]
#         prevframeR2 = [ float(x) for x in zeroFrame[:] ]
        blankframe  = [ float(x) for x in zeroFrame[:] ]
        frameNameL1 = ""
#         frameNameL2 = ""
        frameNameR1 = ""
#         frameNameR2 = ""
        
        # Speaker { windows { ... } }
        # still is faem0_si1392, bug !
        
#         speaker_data = []
#         for sample in monoTrains:
#             frames_info = sample.split('_')
#             if frames_info[0] not in speaker_data:
#                 speaker_data.append(frames_info[0])

#         speaker_data = np.array(speaker_data)
#         speaker_data = np.random.permutation(speaker_data)
# #         print speaker_data
#         valid = len(speaker_data)*0.9
#         speaker_train = speaker_data[ 0 : valid ]
#         speaker_valid = speaker_data[ valid : ]
        
        speaker_train = []
        speaker_valid = []
        with open(MFCC_TRAINING_SPEAKER_ARK) as f:
            for lines in f:
                frames = lines.split('\n')
                speaker_train.append(frames[0])
        with open(MFCC_VALID_SPEAKER_ARK) as f:
            for lines in f:
                frames = lines.split('\n')
                speaker_valid.append(frames[0])

    
        for sample in monoTrains:
            frames_info = sample.split('_')
            speaker = frames_info[0] + "_" + frames_info[1]
#             frameNameL2 = frames_info[0] + "_" + frames_info[1] + "_" + str( (int(frames_info[2])-2))
            frameNameL1 = frames_info[0] + "_" + frames_info[1] + "_" + str( (int(frames_info[2])-1))
            frameNameR1 = frames_info[0] + "_" + frames_info[1] + "_" + str( (int(frames_info[2])+1))
#             frameNameR2 = frames_info[0] + "_" + frames_info[1] + "_" + str( (int(frames_info[2])+2))
            
            if frames_info[0] in speaker_train:
                
                if trains.get(speaker) is None:
                    trains[speaker] = {}
                if   frames_info[2] == "1":
                    trains[speaker][sample] = blankframe + monoTrains[sample] + monoTrains[frameNameR1]
                elif monoTrains.get(frameNameR1) is None:
                    trains[speaker][sample] = monoTrains[frameNameL1] + monoTrains[sample] + blankframe
                else:
                    trains[speaker][sample] = monoTrains[frameNameL1] + monoTrains[sample] + monoTrains[frameNameR1]  
            else:
                if valids.get(speaker) is None:
                    valids[speaker] = {}
                if   frames_info[2] == "1":
                    valids[speaker][sample] = blankframe + monoTrains[sample] + monoTrains[frameNameR1]

                elif monoTrains.get(frameNameR1) is None:
                    valids[speaker][sample] = monoTrains[frameNameL1] + monoTrains[sample] + blankframe

                else:
                    valids[speaker][sample] = monoTrains[frameNameL1] + monoTrains[sample] + monoTrains[frameNameR1]

        monoTrains.clear()
        
#         print len(trains)
#         print len(valids)
        
        for sample in trains:
#             print sample
#             print len(trains[sample])
            
            if sample in sentense:
                temp = []
                for i in range(len(trains[sample])):
                    temp.append( ( trains[sample][sample + "_" + str(i+1)]) ) 

                x_train.append( floatX(temp ))
#                 y_train.append( np.array(sentense[sample]).astype(np.int32) )
                y_train.append( floatX(sentense[sample]) )
                if ( len(temp) < len(sentense[sample]) ):
                    print "wtf", sample
        
        for sample in valids:
#             print sample
#             print len(trains[sample])
            if sample in sentense:
                temp = []
                for i in range(len(valids[sample])):
                    ( temp.append(  ( valids[sample][sample + "_" + str(i+1)]) ))
                x_valid.append( floatX(temp ))
#                 y_valid.append( np.array(sentense[sample]).astype(np.int32) )
                y_valid.append( floatX(sentense[sample]) )
                
                if ( len(temp) < len(sentense[sample]) ):
                    print "wtf", sample
    
#     print "normalized"
#     x_train = preprocessing.scale( x_train)
#     x_valid = preprocessing.scale( x_valid)
    
    trains.clear()
    valids.clear()
        
    return char_map, char_unmap, x_train, y_train, x_valid, y_valid

def floatX(X):
    return np.asarray(X, dtype=theano.config.floatX)

def vectorized_result ( j , siz) :
    e = np.zeros((siz, 1))
    e[j] = 1.0
    return np.reshape( e, siz)


In [4]:
char_map, char_unmap, x_train, y_train, x_valid, y_valid = DNNDataset("mfcc")

mcth0_sx219


In [9]:
print y_train[0]
# x_train[0][0].shape[0]
print (char_unmap)
print len(char_unmap)

print len(x_train)
print len(x_train[0])
print x_train[0][0].shape

[  9.  12.   2.   7.  18.   0.   3.  15.   4.   1.  14.   7.  16.   3.  11.
  16.   1.   0.  11.   2.   7.   4.   3.  11.  17.  12.   8.  20.   6.   1.
   2.   7.   4.  10.   8.  12.   8.  20.   6.   1.   2.   7.  10.  11.   6.
   7.  10.  11.  17.  20.   4.  10.   0.   7.   5.   1.   4.   3.  16.  12.
   8.  12.   1.   2.]
{0: 'r', 1: 'e', 2: 's', 3: 'o', 4: 'l', 5: 'v', 6: 'd', 7: ' ', 8: 't', 9: 'h', 10: 'a', 11: 'n', 12: 'i', 13: 'y', 14: 'm', 15: 'b', 16: 'c', 17: 'g', 18: 'p', 19: 'f', 20: 'u', 21: 'w', 22: 'k', 23: "'", 24: 'q', 25: 'x', 26: 'z', 27: 'j', 28: '-'}
29
3319
384
(117,)


In [8]:
# print x_train.shape
# print x_train[0].shape
# print x_train[0][1]

In [ ]:
print x_test[0].shape
print x_test[0][1][0]

# Data part

## make batch

In [5]:
def make_batches_X(X, length, batch_size=30):
    '''
    https://github.com/craffel/lstm_benchmarks/blob/master/lasagne/experiment.py
    
    Convert a list of matrices into batches of uniform length
    :parameters:
        - X : list of np.ndarray
            List of matrices
        - length : int
            Desired sequence length.  Smaller sequences will be padded with 0s,
            longer will be truncated.
        - batch_size : int
            Mini-batch size
    :returns:
        - X_batch : np.ndarray
            Tensor of time series matrix batches,
            shape=(n_batches, length, batch_size, n_features)
        - X_mask : np.ndarray
            shape=(n_batches, length, batch_size)
            Mask denoting whether to include each time step of each time series
            matrix
    '''
    n_batches = len(X)//batch_size
    
    # Lasagne format
    X_batch = np.zeros((n_batches, batch_size, length, X[0].shape[1]),
                       dtype=theano.config.floatX)
    # Else format
#     X_batch = np.zeros((n_batches, length, batch_size, X[0].shape[1]),
#                        dtype=theano.config.floatX)
    
#     X_mask = np.zeros(X_batch.shape, dtype=np.bool)
    
    X_mask = np.zeros((n_batches, length, batch_size ), dtype=theano.config.floatX)
#     X_mask = np.zeros((n_batches,  batch_size, length ), dtype=theano.config.floatX)

    
    
    for b in range(n_batches): 
        for n in range(batch_size): # go thorough batch size
            
            X_m = X[b*batch_size + n] # seq_length X feature dim
            
            X_batch[b, n, :X_m.shape[0]] = X_m[:length]
#             X_batch[b, :X_m.shape[0], n] = X_m[:length]
            
#             X_mask[b, n, :X_m.shape[0]] = 1
            X_mask[b, :X_m.shape[0], n] = 1
            
    return X_batch, X_mask

def make_batches_Y( X, length, batch_size=30):
    '''
    https://github.com/craffel/lstm_benchmarks/blob/master/lasagne/experiment.py
    
    Convert a list of matrices into batches of uniform length
    :parameters:
        - X : list of np.ndarray
            List of matrices
        - length : int
            Desired sequence length.  Smaller sequences will be padded with 0s,
            longer will be truncated.
        - batch_size : int
            Mini-batch size
    :returns:
        - X_batch : np.ndarray
            Tensor of time series matrix batches,
            shape=(n_batches, length, batch_size )
        - X_mask : np.ndarray
            Mask denoting whether to include each time step of each time series
            matrix
    '''
    n_batches = len(X)//batch_size
    
    X_batch = np.zeros( (n_batches, length, batch_size ), dtype='float32')
    
    X_mask = np.zeros(X_batch.shape, dtype=theano.config.floatX)
    
    for b in range(n_batches):
        for n in range(batch_size):
            X_m = X[ b*batch_size + n ]
            X_batch[b, :X_m.shape[0], n ] = X_m[:length]
            X_mask[b, :X_m.shape[0], n] = 1
    return X_batch, X_mask


In [6]:
batch_size = 40

# Find the longest sequence
length_x = max(max([X.shape[0] for X in x_train]),
             max([X.shape[0] for X in x_valid]))
length_y = max(max([X.shape[0] for X in y_train]),
             max([X.shape[0] for X in y_valid]))

# Convert to batches of time series of uniform length
# x_train_mask: seq_length X batch_size
x_train, x_train_mask = make_batches_X(x_train, length_x, batch_size)
# y_train_mask: output_length X batch_size
# y_pred_mask = x_train_mask, since pred by sequence
y_train, y_train_mask = make_batches_Y(y_train, length_y, batch_size)
x_valid, x_valid_mask = make_batches_X(x_valid, length_x, batch_size)
y_valid, y_valid_mask = make_batches_Y(y_valid, length_y, batch_size)

# x_train_t, x_train_mask = make_batches_X(x_train, length_x, batch_size)
# # y_train_mask: output_length X batch_size
# # y_pred_mask = x_train_mask, since pred by sequence
# y_train_t, y_train_mask = make_batches_Y(y_train, length_y, batch_size)
# x_valid_t, x_valid_mask = make_batches_X(x_valid, length_x, batch_size)
# y_valid_t, y_valid_mask = make_batches_Y(y_valid, length_y, batch_size)


In [7]:
print x_train.shape
print y_train.shape
print x_valid.shape
print y_valid.shape
print "================mask================"
print x_train_mask.shape
print y_train_mask.shape

# print x_train_mask[0][400][0]

(82, 40, 777, 117)
(82, 79, 40)
(9, 40, 777, 117)
(9, 79, 40)
================mask================
(82, 777, 40)
(82, 79, 40)


In [ ]:
a = np.array([1,2,3,4,5])
b = np.array([1,0,0,0,1])
a[np.nonzero(b)]

In [ ]:
# y_valid_mask[0].flatten().reshape( y_valid_t.shape[2], y_valid_t.shape[1])
# print y_train_mask[0]
mask = np.swapaxes( y_train_mask[0], 0, 1)[0]
ans = np.swapaxes( y_train_t[0], 0, 1)[0]

for i in xrange(batch_size):
    mask = np.swapaxes( y_train_mask[0], 0, 1)[i]
    ans = np.swapaxes( y_train_t[0], 0, 1)[i]
    print remap_back( ans[np.nonzero(mask)] )

In [ ]:
for i in xrange(batch_size):
    mask = np.swapaxes( y_train_mask[0], 0, 1)[i]
    ans = np.swapaxes( y_train_t[0], 0, 1)[i]
    print remap_back( ans[np.nonzero(mask)] )

In [ ]:
remap_back( y_valid[0])

# Data part

## decode

In [8]:
'''
credit by NLTK package of measure edit distance
'''

def _edit_dist_init(len1, len2):
    lev = []
    for i in range(len1):
        lev.append([0] * len2)  # initialize 2D array to zero
    for i in range(len1):
        lev[i][0] = i           # column 0: 0,1,2,3,4,...
    for j in range(len2):
        lev[0][j] = j           # row 0: 0,1,2,3,4,...
    return lev


def _edit_dist_step(lev, i, j, s1, s2, transpositions=False):
    c1 = s1[i - 1]
    c2 = s2[j - 1]

    # skipping a character in s1
    a = lev[i - 1][j] + 1
    # skipping a character in s2
    b = lev[i][j - 1] + 1
    # substitution
    c = lev[i - 1][j - 1] + (c1 != c2)

    # transposition
    d = c + 1  # never picked by default
    if transpositions and i > 1 and j > 1:
        if s1[i - 2] == c2 and s2[j - 2] == c1:
            d = lev[i - 2][j - 2] + 1

    # pick the cheapest
    lev[i][j] = min(a, b, c, d)


def check_label_error( real , predict, transpositions=False):
    ## length of real >= length of predict
    """
    Calculate the Levenshtein edit-distance between two strings.
    The edit distance is the number of characters that need to be
    substituted, inserted, or deleted, to transform s1 into s2.  For
    example, transforming "rain" to "shine" requires three steps,
    consisting of two substitutions and one insertion:
    "rain" -> "sain" -> "shin" -> "shine".  These operations could have
    been done in other orders, but at least three steps are needed.

    This also optionally allows transposition edits (e.g., "ab" -> "ba"),
    though this is disabled by default.

    :param s1, s2: The strings to be analysed
    :param transpositions: Whether to allow transposition edits
    :type s1: str
    :type s2: str
    :type transpositions: bool
    :rtype int
    """
    # set up a 2-D array
    len1 = len(predict)
    len2 = len(real)
    lev = _edit_dist_init(len1 + 1, len2 + 1)

    # iterate over the array
    for i in range(len1):
        for j in range(len2):
            _edit_dist_step(lev, i + 1, j + 1, predict, real, transpositions=transpositions)
            
    return lev[len1][len2]*1.0/len2


def clean_up( y ):
    """
    for final output clean up
    B(a − ab−) = B(−aa − −abb) = aab
    """
    answer = ""

    for i in range(len(y)):
        if y[i] == '-':
            continue
        else:
            if y[i-1] != y[i]:
                answer += y[i]
    
    return answer

def remap_back( y , actual):
    answer = ""
    
    for i in y:
        answer += char_unmap[i]
    
    if not actual:
        answer = clean_up(answer)
        
    return answer

def decode_all_actual( y, y_mask ,batch_size , actual=False):
    """
    y     : label_length X batch_size
    y_mask: label_length X batch_size
    """
    result = []
    for i in xrange(batch_size):
        mask = np.swapaxes( y_mask, 0, 1)[i]
        ans = np.swapaxes( y, 0 , 1)[i]
        result.append(remap_back( ans[np.nonzero(mask)] , actual ))
    return result


def decode_all_pred( y, y_mask ,batch_size , actual=False):
    """
    y     : label_length X batch_size
    y_mask: label_length X batch_size
    """
    result = []
    for i in xrange(batch_size):
        mask = np.swapaxes( y_mask, 0, 1)[i]
        ans = y[i]
        result.append(remap_back( ans[np.nonzero(mask)] , actual ))
    return result

def check_all( y, y_mask, y_pred, y_pred_mask, batch_size ):
    
    actual = decode_all_actual( y, y_mask, batch_size , True )
    predict = decode_all_pred( y_pred, y_pred_mask, batch_size , False )
    
    error = 0.
    for a,b in zip (actual, predict):
        error += check_label_error(a,b)
    
    return error/len(actual)

In [43]:
# remap_back( y_train[0])
# clean_up( "-a c--abbaabb- ddf" )
pred = decode_all_actual( y_train[0], y_train_mask[0], batch_size , True)
pred

['his problem concerns longitudes latitudes and angular velocities',
 'how much allowance do you get',
 "boy you're stirrin early a sleepy voice said",
 'why put such a high value on being top dog',
 'are holiday aprons available to us',
 'encyclopedias seldom present anecdotal evidence',
 'eat your raisins outdoors on the porch steps',
 'how much allowance do you get',
 'ducks have webbed feet and colorful feathers',
 'we could barely see the fjords through the snow flurries',
 'critical equipment needs proper maintenance',
 'the mango and the papaya are in a bowl',
 'both have excellent integration of their fiscal tax collection year calendars',
 'doctors prescribe drugs too freely',
 'we are open every monday evening',
 'the frightened child was gently subdued by his big brother',
 'tofu is made from processed soybeans',
 "a moth zig zagged along the path through otto's garden",
 'remove the splinter with a pair of tweezers',
 'todd placed top priority on getting his bike fixed',
 '

In [ ]:
a = "I am a boy"
b = "I m a boy"

check_label_error(a,b)


In [22]:
abc = model1.train(x_train[0], y_train[0], x_train_mask[0], y_train_mask[0])

In [97]:
abc = model2.predict(x_train[0])
len(abc)
abc[0].shape

(40, 777, 29)

In [98]:
# remap_back( (decode_all( np.argmax(abc[2],axis=2) , x_train_mask[0],  batch_size , True)[0]), True)
( decode_all_pred( np.argmax(abc[0],axis=2) , x_train_mask[0],  batch_size , False))

['his plm concerns longitdes litds nd anglor velocitee',
 'how mouch alwoance o you get',
 'boy your strrin early a sled voice id',
 'why puit sucha hig valu on being top doe',
 'ar holiday aprons available to us',
 'enyclopedias sel dom presnt anct evidenc',
 'eat yor reasins odors on the porch stes',
 'o muc lowance do o get',
 'ducks have whbd fe n coorul feathers',
 'we coud barely se the fjords trot the snow frise',
 'critical euitpment neds proper mitenance',
 'the mang and he papy are in a bol',
 'tbo hve axcln intgaion o thie fiscal tax clictiony ear clndars',
 'doktors prscrib drug to frely',
 'e ae en evrey mnd eening',
 'the frhene hild was ently subded bhis big brother',
 'tof is made from progcessed soiybeans',
 'a moth zi zged ong the pat thregh tos gardn',
 'rem the splinter wih a pi f tweers',
 'tod placed to prrity on geting his bike fixed',
 'ust drop notoes in an sagstn box',
 "barb's gld bracelet was  gragduation prsnt",
 'tecthnical writers can abrevite in biblogra

In [59]:
# check_all( y, y_mask, y_pred, y_pred_mask, batch_size ):
check_all( y_train[0], y_train_mask[0], np.argmax(abc[0],axis=2),  x_train_mask[0] ,batch_size)

0.2523178379406231

# Model part

In [9]:
# def relu(X):
#     # return T.maximum(X, 0.)
# #     return lambda x: x * (x > 0)
#     return T.minimum(X * (X > 0), 20)

def relu(x):
    return x * (T.lt(x, 1e-6) and T.gt(x, 20))

def build_shared_zeros(shape):
    """ Builds a theano shared variable filled with a zeros numpy array """
    return theano.shared(value=np.zeros(shape, dtype=theano.config.floatX),
             borrow=True)

def dropout(random_state, X, keep_prob=0.5):
    if keep_prob > 0. and keep_prob < 1.:
        seed = random_state.randint(2 ** 30)
        srng = RandomStreams(seed)
        mask = srng.binomial(n=1, p=keep_prob, size=X.shape,
                             dtype=theano.config.floatX)
        return X * mask
    return X


def fast_dropout(random_state, X):
    seed = random_state.randint(2 ** 30)
    srng = RandomStreams(seed)
    mask = srng.normal(size=X.shape, avg=1., dtype=theano.config.floatX)
    return X * mask


# momentum method
def momentum(loss, all_params, update_momentum=0.9, update_learning_rate=0.1, nesterov=True):
    all_grads = theano.grad(loss, all_params)
    updates = []

    for param_i, grad_i in zip(all_params, all_grads):
        mparam_i = theano.shared(np.zeros(param_i.get_value().shape,
                                          dtype=theano.config.floatX),
                                 broadcastable=param_i.broadcastable)
        v = update_momentum * mparam_i - update_learning_rate * grad_i
        updates.append((mparam_i, v))
        
        if nesterov:
            new_p = param_i + update_momentum * v - update_learning_rate* grad_i
        else:
            new_p = param_i + v
            
        updates.append((param_i, new_p ))

    return updates




def build_rnn_combine( X, num_batch, W_input_hidden1, b_hidden1, 
                       W_hidden1_hidden2, b_hidden2, 
                       W_hidden2_hidden3, b_hidden3,
                       W_hidden3_rnnhidden, 
                       W_rnnhidden_rnnhidden, b_rnnhidden,
                       W_rnnhidden_output, i_rnnhidden, b_output ):

    def step(input_curr,hidden_prev):
        
        # (batch X 117) dot (117 X hidden1)
        hidden1 = relu( T.dot( input_curr , W_input_hidden1) + b_hidden1 )
        # (batch X hidden1) dot ( hidden1 X hidden2 )
        hidden2 = relu( T.dot( hidden1 , W_hidden1_hidden2) + b_hidden2 )
        # (batch X hidden2) dot ( hidden2 X hidden3 )
        hidden3 = relu( T.dot( hidden2 , W_hidden2_hidden3) + b_hidden3 )
        
        # (batch X hidden3) dot ( hidden3 X rnnhidden1 ) + 
        # (batch X rnnhidden1) dot ( rnnhidden1 X rnnhidden1 ) 
        hidden_rnn = relu( T.dot(hidden3, W_hidden3_rnnhidden ) + 
                      T.dot( hidden_prev, W_rnnhidden_rnnhidden ) +
                      b_rnnhidden )
        
        # (batch X rnnhidden1) dot ( rnnhidden1 X ( num_class+1) ) 
        y_pred = T.dot( hidden_rnn, W_rnnhidden_output ) + b_output
        
        return hidden_rnn, y_pred
        
    # Iteration over the first dimension of a tensor which is TIME in our case
    # recurrent_fn doesn't use y in the computations, so we do not need y0 (None)
    # scan returns updates too which we do not need. (_)
    [hidden, y_pred] , _ = theano.scan(
        step,
        sequences = X,
        outputs_info = [i_rnnhidden, None],
        truncate_gradient=5
    )
    
    return y_pred


class end_to_end():
    
    def __init__(self, input_shape, max_seq_length, hidden_layer, rnn_hidden_layer,
                 batch, max_epochs, eval_size, output_num_units,
                 patience, up_learning_rate, up_momentum, file_name):
        
        self.input_shape = input_shape # [batch, dim]
        self.max_seq_length = max_seq_length
        self.hidden_layer = hidden_layer # hidden [l1, l2, l3]
        self.rnn_hidden_layer = rnn_hidden_layer # rnn hidden [l1]
        self.output_num_units = output_num_units # [ # of class ]
        
        self.batch = batch
        self.max_epochs = max_epochs
        self.eval_size = eval_size
        
        self.up_learning_rate = up_learning_rate
        self.up_momentum = up_momentum
                
        self.patience = patience
        self.best_valid = np.inf
        self.best_valid_epoch = 0
        self.best_params = None
        
        self.train_history_ = []
        self.epochs = 0
        
        self.file_name = file_name
        
        self.drop_frac = 0.2
#         X = T.matrix('X')
#         Y = T.ivector('Y')     
        
        """
        input data type
        y_hat : T x B x C+1
        y : L x B
        y_hat_mask : T x B
        y_mask : L x B
        """
        
        # T x B x F
        # B X T X F (Lasagne format)
        x = T.tensor3('X', dtype=theano.config.floatX)
        # L x B
        y = T.matrix('y', dtype=theano.config.floatX)

        # L x B
        y_mask = T.matrix('y_mask', dtype=theano.config.floatX)
        # T x B
        y_hat_mask = T.matrix('y_hat_mask', dtype=theano.config.floatX)
    
        # Min/max sequence length
#         MIN_LENGTH = 50
        MAX_LENGTH = max_seq_length
        # Number of units in the hidden (recurrent) layer
        N_HIDDEN = hidden_layer[0]
        # Number of training sequences in each batch
        N_BATCH = batch
        # Optimization learning rate
        GRAD_CLIP = 100
        # How often should we check the output?
        EPOCH_SIZE = 100
        # Number of epochs to train the net
        NUM_EPOCHS = 10
        
        #===========================================================================================
        # Bi RNN
        #===========================================================================================

        # Recurrent layers expect input of shape
        # (batch size, max sequence length, number of features)
        # T x B x F
#         l_in = lasagne.layers.InputLayer(shape=( N_BATCH, MAX_LENGTH,  self.input_shape[1]))
        
        # We're using a bidirectional network, which means we will combine two
        # RecurrentLayers, one with the backwards=True keyword argument.
        # Setting a value for grad_clipping will clip the gradients in the layer
#         l_forward = lasagne.layers.RecurrentLayer(l_in, N_HIDDEN, grad_clipping=GRAD_CLIP,
#                                                   W_in_to_hid=lasagne.init.HeUniform(),
#                                                   W_hid_to_hid=lasagne.init.HeUniform(),
#                                                   nonlinearity=lasagne.nonlinearities.tanh)
#         l_backward = lasagne.layers.RecurrentLayer(l_in, N_HIDDEN, grad_clipping=GRAD_CLIP,
#                                                    W_in_to_hid=lasagne.init.HeUniform(),
#                                                    W_hid_to_hid=lasagne.init.HeUniform(),
#                                                    nonlinearity=lasagne.nonlinearities.tanh, 
#                                                    backwards=True)
        # We'll use an elementwise sum to combine the forward/backward layers
#         l_sum = lasagne.layers.ElemwiseSumLayer([l_forward, l_backward])
        
        # We need a reshape layer which combines the first (batch size) and second
        # (number of timesteps) dimensions, otherwise the DenseLayer will treat the
        # number of time steps as a feature dimension
#         l_reshape = lasagne.layers.ReshapeLayer( l_sum, (N_BATCH*MAX_LENGTH, N_HIDDEN))
        
        #===========================================================================================
        # dEEP lSTM
        #===========================================================================================
        
        # Recurrent layers expect input of shape
        # (batch size, max sequence length, number of features)
        l_in = lasagne.layers.InputLayer(shape=(N_BATCH, MAX_LENGTH,  self.input_shape[1]))
        
        l_in_gau = lasagne.layers.GaussianNoiseLayer(l_in, sigma=0.5 )
        
        # LSTM layer 1
        l_forward_1 = lasagne.layers.LSTMLayer(l_in_gau, num_units=hidden_layer[0], 
                                               learn_init=True, peepholes=True)
        l_backward_1 = lasagne.layers.LSTMLayer(l_in, num_units=hidden_layer[0],
                                                backwards=True, learn_init=True, peepholes=True)
        l_recurrent_1 = ElemwiseSumLayer([l_forward_1, l_backward_1])
  
        # concatenate forward and backward LSTM layers
#         l_fwd_reshape_1 = lasagne.layers.ReshapeLayer(l_forward_1, (N_BATCH*MAX_LENGTH, hidden_layer[0]))
#         l_bck_reshape_1 = lasagne.layers.ReshapeLayer(l_backward_1, (N_BATCH*MAX_LENGTH, hidden_layer[0]))
#         l_recurrent_1 = lasagne.layers.ConcatLayer([l_fwd_reshape_1, l_bck_reshape_1], axis=1)
        
#         if self.drop_frac > 0:
#             l_drp_1 = lasagne.layers.DropoutLayer(l_recurrent_1, p = 0.2 )
#         else:
#             l_drp_1 = l_recurrent_1

        # LSTM layer 2
        l_forward_2 = lasagne.layers.LSTMLayer(l_recurrent_1, num_units=hidden_layer[1], 
                                               learn_init=True, peepholes=True)
        l_backward_2 = lasagne.layers.LSTMLayer(l_recurrent_1, num_units=hidden_layer[1],
                                                backwards=True, learn_init=True, peepholes=True)
        l_recurrent_2 = ElemwiseSumLayer([l_forward_2, l_backward_2])
        
#         if self.drop_frac > 0:
#             l_drp_2 = lasagne.layers.DropoutLayer(l_recurrent_2, p = 0.2 )
#         else:
#             l_drp_2 = l_recurrent_2
        
        # LSTM layer 3
        l_forward_3 = lasagne.layers.LSTMLayer(l_recurrent_2, num_units=hidden_layer[2], 
                                               learn_init=True, peepholes=True)
        l_backward_3 = lasagne.layers.LSTMLayer(l_recurrent_2, num_units=hidden_layer[2],
                                                backwards=True, learn_init=True, peepholes=True)
        l_recurrent_3 = ElemwiseSumLayer( [l_forward_3, l_backward_3] )
        
#         if self.drop_frac > 0:
#             l_drp_3 = lasagne.layers.DropoutLayer(l_recurrent_3, p = 0.2 )
#         else:
#             l_drp_3 = l_recurrent_3

        
        # LSTM layer 4
        l_forward_4 = lasagne.layers.LSTMLayer(l_recurrent_3, num_units=hidden_layer[3], 
                                               learn_init=True, peepholes=True)
        l_backward_4 = lasagne.layers.LSTMLayer(l_recurrent_3, num_units=hidden_layer[3],
                                                backwards=True, learn_init=True, peepholes=True)
        l_recurrent_4 = ElemwiseSumLayer( [l_forward_4, l_backward_4] )

#         if self.drop_frac > 0:
#             l_drp_4 = lasagne.layers.DropoutLayer(l_recurrent_4, p = 0.2 )
#         else:
#             l_drp_4 = l_recurrent_4
            
        # LSTM layer 5
#         l_forward_5 = lasagne.layers.LSTMLayer(l_drp_4, num_units=hidden_layer[4], 
#                                                learn_init=True, peepholes=True)
#         l_backward_5 = lasagne.layers.LSTMLayer(l_drp_4, num_units=hidden_layer[4],
#                                                 backwards=True, learn_init=True, peepholes=True)
#         l_recurrent_5 = ElemwiseSumLayer( [l_forward_5, l_backward_5] )

#         if self.drop_frac > 0:
#             l_drp_5 = lasagne.layers.DropoutLayer(l_recurrent_5, p = 0.2 )
#         else:
#             l_drp_5 = l_recurrent_5


        
        l_reshape = lasagne.layers.ReshapeLayer(l_recurrent_4,
                                               (N_BATCH*MAX_LENGTH, hidden_layer[3]))

        
        #===========================================================================================
        # COMMON SETUP
        #===========================================================================================
        
        # Our output layer is a simple dense connection
        l_recurrent_out = lasagne.layers.DenseLayer( l_reshape, num_units= output_num_units[0] ,
                                                    nonlinearity=lasagne.nonlinearities.identity)
        
        # Now, reshape the output back to the RNN format
        l_out_shp = lasagne.layers.ReshapeLayer(l_recurrent_out, (N_BATCH, MAX_LENGTH, output_num_units[0] ))
        
        # dimshuffle to shape format (input_seq_len, batch_size, num_classes + 1)
        l_out_shp_ctc = lasagne.layers.DimshuffleLayer(l_out_shp, (1, 0, 2))

        l_out_softmax = lasagne.layers.NonlinearityLayer(
                            l_recurrent_out, nonlinearity=lasagne.nonlinearities.softmax)
        l_out_softmax_shp = lasagne.layers.ReshapeLayer(
                            l_out_softmax, (N_BATCH, MAX_LENGTH, output_num_units[0] ))
        
        # since we use gaussian noise in input, False means use noise, True means dont use noise
        output_lin_ctc_train = lasagne.layers.get_output(l_out_shp_ctc, x, deterministic=False)
        output_softmax_train = lasagne.layers.get_output(l_out_softmax_shp, x, deterministic=False)
        
        output_lin_ctc_val = lasagne.layers.get_output(l_out_shp_ctc, x, deterministic=True)
        output_softmax_val = lasagne.layers.get_output(l_out_softmax_shp, x, deterministic=True)
        
        self.all_params = lasagne.layers.get_all_params(l_out_shp)

        # the CTC cross entropy between y and linear output network
        pseudo_cost = ctc_cost_2.pseudo_cost(
            y, output_lin_ctc_train, y_mask, y_hat_mask,
            skip_softmax=True)
        
        
        pseudo_cost_grad = T.grad(pseudo_cost.mean(), self.all_params)
        true_cost = ctc_cost_2.cost(y, output_softmax_train.dimshuffle(1, 0, 2), y_mask, y_hat_mask)
        cost = T.mean(true_cost)
        updates = lasagne.updates.rmsprop(pseudo_cost_grad, self.all_params, learning_rate = self.up_learning_rate)

        self.train = theano.function(
            inputs = [x, y, y_hat_mask, y_mask],
            outputs = [ pseudo_cost.mean(), cost, output_softmax_train ],
#             outputs = [output_lin_ctc, output_softmax, cost],
            updates=updates
        )
        
        self.predict = theano.function( 
            inputs=[x], 
            outputs = [ output_softmax_val] 
        )

        
    # x_mask and y_mask is the same
    # x_test_mask and y_test_mask is the same
    def fit(self, x_train, y_train, x_test,  y_test , x_mask, y_mask, x_test_mask, y_test_mask ):
        print " "
        print "start training!!!!"
        print " "
        
        for i in range(self.max_epochs):
            self.epochs +=1
            t0 = time()
            
            cs = 0.
            pseudo_cs = 0.
            for index in range(len(x_train)):
                pseudo, cost, output_softmax = self.train( x_train[index] , y_train[index],
                                                          x_mask[index], y_mask[index])
                cs += cost
                pseudo_cs += pseudo
                gg = index
#                 print pseudo, cost
                
            cs /= len(x_train)
            pseudo_cs /= len(x_train)
            
#             cPickle.dump( self , open("./"+self.file_name+".pkl","wb"))
            if self.epochs <= 1:
                previous_cs = "-"
            else:
                previous_cs = self.train_history_[-1]['cost']
            print "\n==============================="
            print 'epoch {0} : pseudo= {1}, cost= {2}, previous_cost= {3}, train_time = {4} s'.format(self.epochs,
                                                                                                      pseudo_cs, 
                                                                                                      cs,
                                                                                                      previous_cs, 
                                                                                                      time() - t0)
            
            sys.stdout.flush()
            
            t0 = time()
            
#             print np.argmax(output_softmax[:],axis=2)
#             print decode_all( np.argmax(output_softmax[:],axis=2) , x_train_mask[gg],  batch_size , True)
            
            # save model first
            # dont know the LER performance yet
#             cPickle.dump( self , open("./"+self.file_name+".pkl","wb"))
    

            label_error_rate = 0.
            for index in range( len(x_test) ) :

                prepre = self.predict( x_test[index])
                
                label_error_rate += check_all( y_test[index], y_test_mask[index],
                                              np.argmax(prepre[0],axis=2) , x_test_mask[index],
                                              batch_size)
                
                if (( index + self.epochs ) % 4 == 0 ):
                    ## print actual
                    y_actual = decode_all_actual( y_test[index], y_test_mask[index], batch_size , True)

                    ## print pred
                    y_predict = decode_all_pred( np.argmax(prepre[0],axis=2) , x_test_mask[index],  batch_size , False)
                
            label_error_rate /= len(x_test)
            self.train_history_.append({"epoch":self.epochs, "cost": cs, "LER":label_error_rate})
            
            print '\t\t\t val_= {0}, test_time  = {1} s'.format(label_error_rate, time() - t0)
            print "===============================\n"
            
            sys.stdout.flush()
            
            for a , b in zip (y_actual,y_predict):
                print "Target== ",a 
                print "\tAns => ",b
            
            sys.stdout.flush()
            
            """
            should use cost do early stopping
            """
            current_cs = self.train_history_[-1]['cost']
            current_epoch = self.train_history_[-1]['epoch']
            if current_cs < self.best_valid:
                self.best_valid = current_cs
                self.best_valid_epoch = current_epoch
                self.best_params = [w.get_value() for w in self.all_params]
                cPickle.dump( self , open("./"+self.file_name+".pkl","wb"))
                
            elif self.best_valid_epoch + self.patience <= current_epoch:
                print ""
                print "Early stopping."
                print self.best_valid_epoch,self.best_valid
                print "Best valid ler {:.6f} at epoch {}.".format(self.best_valid, self.best_valid_epoch)              
#                 for qq in range (len(self.all_params)):
#                     self.all_params[qq].set_value( self.best_params[qq] )
#                 break

            """
            can not use label error rate do early stopping
            """
#             current_ler = self.train_history_[-1]['LER']
#             current_epoch = self.train_history_[-1]['epoch']
#             if current_ler < self.best_valid:
# #                 print "********************* Now best ************************"
#                 sys.stdout.flush()
#                 self.best_valid = current_ler
#                 self.best_valid_epoch = current_epoch
#                 self.best_params = [w.get_value() for w in self.all_params]
# #                 cPickle.dump( self , open("./"+self.file_name+".pkl","wb"))
                
#             elif self.best_valid_epoch + self.patience <= current_epoch:
#                 print ""
#                 print "Early stopping."
#                 print self.best_valid_epoch,self.best_valid
#                 print "Best valid ler {:.6f} at epoch {}.".format(self.best_valid, self.best_valid_epoch)
#                 sys.stdout.flush()                
#                 for qq in range (len(self.all_params)):
#                     self.all_params[qq].set_value( self.best_params[qq] )
#                 break


#     def prediction(self, x, x_mask) :
        
#         abc =  self.predict(x, x_mask)
        
#         return np.argmax(abc[0], axis = 1 )



# Experiment part

In [10]:
num_classes = len(char_unmap) - 2
blank_symbol = len(char_unmap)- 1
print num_classes
print blank_symbol

27
28


# Model 3 testing

## with validation data set, with proper predict

### set model 2 result to model 3 first

In [103]:
model3 = end_to_end (
    input_shape      = (1, x_train.shape[3] ) , # batch of 1, (110, 30, 777, 117)
#     input_shape = (1, 117),
    max_seq_length = x_train.shape[2],
    hidden_layer     = [ 200, 200 , 200 ], # maximum layer to LSTM 3 layer only
    rnn_hidden_layer = [ 100 ], # maximum layer to 1 layer only
    batch            = batch_size, 
    max_epochs       = 300, 
    eval_size        = 0.1, 
    output_num_units = [ len(char_unmap) ],
    up_learning_rate = 0.0001, 
    up_momentum      = 0.9, 
    patience         = 5,
    file_name = "model10"
)


In [105]:
for qq in range (len(model2.all_params)):
    model3.all_params[qq].set_value( model2.all_params[qq].get_value() )


### Start training model 2 with new design of validation

In [ ]:
%time model3.fit( x_train, y_train, x_valid, y_valid, x_train_mask, y_train_mask , x_valid_mask, y_valid_mask )

# Model 2 testing

## with validation data set

### set model 1 result to model 2 first

In [94]:
model2 = end_to_end (
    input_shape      = (1, x_train.shape[3] ) , # batch of 1, (110, 30, 777, 117)
#     input_shape = (1, 117),
    max_seq_length = x_train.shape[2],
    hidden_layer     = [ 200, 200 , 200 ], # maximum layer to LSTM 3 layer only
    rnn_hidden_layer = [ 100 ], # maximum layer to 1 layer only
    batch            = batch_size, 
    max_epochs       = 300, 
    eval_size        = 0.1, 
    output_num_units = [ len(char_unmap) ],
    up_learning_rate = 0.0001, 
    up_momentum      = 0.9, 
    patience         = 5,
    file_name = "model9"
)


In [95]:
for qq in range (len(model1.all_params)):
    model2.all_params[qq].set_value( model1.all_params[qq].get_value() )


### continue training model 1 due to connection error

In [ ]:

%time model1.fit( x_train, y_train, x_valid, y_valid, x_train_mask, y_train_mask , x_valid_mask, y_valid_mask )

 
start training!!!!
 

epoch 59 : pseudo= 233.638573164, cost= inf, previous_cost= 37.6649920766, train_time = 1156.38262415 s
			 val_= 0.886226385164, test_time  = 7.91437888145 s

Target==  smash lightbulbs and their cash value will diminish to nothing
	Ans =>  ccshshshshshsa
Target==  which long article was opaque and needed clarification
	Ans =>  ph
Target==  my ideal morning begins with hot coffee
	Ans =>  mw
Target==  necessary retouching was put on at once
	Ans =>  ns asorirteting as putonn wns
Target==  draw every outer line first then fill in the interior
	Ans =>  cc
Target==  did dad do academic bidding
	Ans =>  spsp
Target==  brush fires are common in the dry underbrush of nevada
	Ans =>  resaiirsour conn tedrnder breshod e otn
Target==  maybe twenty thirty fifty
	Ans =>  phphphph
Target==  approach your interview with statuesque composure
	Ans =>  pwpwpwnwnwnw
Target==  it was not whatever tale was told by tails
	Ans =>  i weis not etever tar was tor b tailes
Target==  th

# =================================================

# Model 1 testing

## train output:  pseudo_cost.mean(), cost, output_softmax

In [ ]:
model1 = end_to_end (
    input_shape      = (1, x_train.shape[3] ) , # batch of 1, (110, 30, 777, 117)
#     input_shape = (1, 117),
    max_seq_length = x_train.shape[2],
    hidden_layer     = [ 250, 250 , 250, 250 ], # maximum layer to LSTM 3 layer only
    rnn_hidden_layer = [ 100 ], # maximum layer to 1 layer only
    batch            = batch_size, 
    max_epochs       = 300, 
    eval_size        = 0.1, 
    output_num_units = [ len(char_unmap) ],
    up_learning_rate = 0.1, 
    up_momentum      = 0.9, 
    patience         = 5,
    file_name = "model_11_char_1"
)


/home/pika/nntools/lasagne/layers/helper.py:69: UserWarning: get_all_layers() has been changed to return layers in topological order. The former implementation is still available as get_all_layers_old(), but will be removed before the first release of Lasagne. To ignore this warning, use `warnings.filterwarnings('ignore', '.*topo.*')`.
  warnings.warn("get_all_layers() has been changed to return layers in "
/home/pika/anaconda/lib/python2.7/site-packages/theano/scan_module/scan.py:1017: Warning: In the strict mode, all neccessary shared variables must be passed as a part of non_sequences
  'must be passed as a part of non_sequences', Warning)


In [13]:
model1.train_history_

[{'LER': 0.9999455337690633, 'cost': 176.20454648645912, 'epoch': 1},
 {'LER': 0.9998366013071895, 'cost': 142.81668779326648, 'epoch': 2},
 {'LER': 0.9998366013071895, 'cost': 133.21784675412061, 'epoch': 3},
 {'LER': 0.9954028464546139, 'cost': 145.9776240092952, 'epoch': 4},
 {'LER': 0.9998546113847837, 'cost': 146.03429245367283, 'epoch': 5},
 {'LER': 0.9994892284700571, 'cost': 123.14759342844893, 'epoch': 6},
 {'LER': 0.9917191596940014, 'cost': 121.69835495367283, 'epoch': 7},
 {'LER': 0.993915001515576, 'cost': 118.9793646277451, 'epoch': 8},
 {'LER': 0.9744307072192717, 'cost': 113.70241704801234, 'epoch': 9},
 {'LER': 0.9228201717245942, 'cost': 108.38464262427354, 'epoch': 10},
 {'LER': 0.8727995673340172, 'cost': 102.87437206361352, 'epoch': 11},
 {'LER': 0.8266841751576175, 'cost': 98.763681458263861, 'epoch': 12},
 {'LER': 0.8073575309640262, 'cost': 95.849358907560017, 'epoch': 13},
 {'LER': 0.7418083664065973, 'cost': 94.287847658482988, 'epoch': 14},
 {'LER': 0.8004275

In [ ]:
%time model1.fit( x_train, y_train, x_valid, y_valid, x_train_mask, y_train_mask , x_valid_mask, y_valid_mask )

 
start training!!!!
 

epoch 22 : pseudo= -5.71634824356, cost= 86.8162095605, previous_cost= 94.6620867659, train_time = 1142.47814798 s
			 val_= 0.673180318628, test_time  = 8.15387296677 s

Target==  by eating yogurt you may live longer
	Ans =>  t  mnt pnr
Target==  fill that canteen with fresh spring water
	Ans =>  l ttwi frsprr
Target==  the same shelter could be built into an embankment or below ground level
	Ans =>  sr o be bont n b r b gr
Target==  they're doin' it now
	Ans =>  her wt 
Target==  he may have a point in urging that decadent themes be given fewer prizes
	Ans =>  m  p ra tc s e n r piis
Target==  that stinging vapor was caused by chloride vaporization
	Ans =>  sanmper ws c br prsion
Target==  continental drift is a geological theory
	Ans =>  connr tds  clther
Target==  a huge tapestry hung in her hallway
	Ans =>   t hstr ner
Target==  nothing is as offensive as innocence
	Ans =>  nmfis  s  nss  snss
Target==  the barracuda recoiled from the serpent's poisonous fa

In [ ]:
abc = model1.train(x_train[0], y_train[0], x_train_mask[0], y_train_mask[0])

In [ ]:
# pseudo_cost.mean(), cost, output_softmax 

# decode_all( y_train[0], y_train_mask[0], batch_size, True)
len(decode_all( np.argmax(abc[2],axis=2) , x_train_mask[0],  batch_size , True)[0])


In [15]:
cPickle.dump( model1 , open("./model_11_char_1.pkl","wb"))

# load

In [12]:
import gc
gc.collect()

del model1

with open('model_11_char_1.pkl', 'rb') as f:
    model1 = cPickle.load(f)


/home/pika/anaconda/lib/python2.7/site-packages/theano/scan_module/scan_perform_ext.py:133: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility
  from scan_perform.scan_perform import *
